<a href="https://colab.research.google.com/github/ben854719/DriveSmart-An-Autonomous-Mobility-Solution/blob/main/Remote_Starter_Bluetooth_Application.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import socket

# Replace with your actual IP address and port
TARGET_HOST = '192.168.1.100'
TARGET_PORT = 12345

try:
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    client_socket.connect((TARGET_HOST, TARGET_PORT))

    message = "Hello from Colab!"
    client_socket.sendall(message.encode())

    # Optional: receive response
    response = client_socket.recv(1024)
    print("Received:", response.decode())

    client_socket.close()
except Exception as e:
    print(f"Connection error: {e}")


Connection error: [Errno 110] Connection timed out


In [3]:
!sudo apt-get install bluetooth libbluetooth-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  bluez libbluetooth3 libudev1 systemd-hwe-hwdb udev
Suggested packages:
  bluez-cups bluez-obexd bluez-meshd pulseaudio-module-bluetooth
The following NEW packages will be installed:
  bluetooth bluez libbluetooth-dev libbluetooth3 systemd-hwe-hwdb udev
The following packages will be upgraded:
  libudev1
1 upgraded, 6 newly installed, 0 to remove and 34 not upgraded.
Need to get 3,087 kB of archives.
After this operation, 14.7 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libudev1 amd64 249.11-0ubuntu3.16 [76.7 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 udev amd64 249.11-0ubuntu3.16 [1,557 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 bluez amd64 5.64-0ubuntu1.4 [1,106 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates/univers

In [4]:
!pip install bleak

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 927.3/927.3 kB 30.4 MB/s eta 0:00:00


In [5]:
from unittest import mock
import asyncio
from bleak import BleakScanner

# Fake device data
fake_devices = [
    mock.Mock(name="Device1", address="AA:BB:CC:DD:EE:FF"),
    mock.Mock(name="Device2", address="11:22:33:44:55:66"),
]

# Patch the discover method
@mock.patch('bleak.BleakScanner.discover', return_value=fake_devices)
def test_bluetooth_scan(mock_discover):
    devices = asyncio.run(BleakScanner.discover())
    for d in devices:
        print(f"Found device: {d.name} @ {d.address}")

In [6]:
from bleak import BleakClient

async def send_command(address, command):
    async with BleakClient(address) as client:
        await client.write_gatt_char("command-char-uuid", command.encode())

In [8]:
from unittest.mock import patch, AsyncMock
from unittest import mock

@patch('your_module.BleakClient', new_callable=AsyncMock)
def test_failed_write(self, mock_client_class):
    mock_client = mock_client_class.return_value.__aenter__.return_value

    # Simulate write_gatt_char throwing an exception
    mock_client.write_gatt_char.side_effect = Exception("Write failed")

    address = "AA:BB:CC:DD:EE:FF"
    command = "STOP"

    import asyncio
    with self.assertRaises(Exception) as context:
        asyncio.run(send_command(address, command))

    self.assertEqual(str(context.exception), "Write failed")

In [9]:
from bleak import BleakScanner

async def find_target_device(target_name):
    devices = await BleakScanner.discover()
    for d in devices:
        if d.name == target_name:
            return d.address
    return None

In [20]:
import unittest
from unittest.mock import patch, Mock
import asyncio
from bleak import BleakScanner
import nest_asyncio

# Apply nest_asyncio to allow nested event loops
nest_asyncio.apply()

# The find_target_device function is defined in a previous cell,
# so we don't need to import it from a module.

class TestBluetoothScanner(unittest.TestCase):
    @patch('bleak.BleakScanner.discover')
    async def test_device_found(self, mock_discover):
        # Configure the mock object to return the desired name and address
        fake_device = Mock()
        fake_device.configure_mock(name="MyDevice", address="DE:AD:BE:EF:00:01")
        mock_discover.return_value = [fake_device]

        address = await find_target_device("MyDevice")
        self.assertEqual(address, "DE:AD:BE:EF:00:01")

    @patch('bleak.BleakScanner.discover')
    async def test_device_not_found(self, mock_discover):
        mock_discover.return_value = []

        address = await find_target_device("GhostDevice")
        self.assertIsNone(address)

# Run the tests manually
if __name__ == '__main__':
    tests = TestBluetoothScanner()
    print("Running test_device_found...")
    asyncio.run(tests.test_device_found())
    print("Running test_device_not_found...")
    asyncio.run(tests.test_device_not_found())

Running test_device_found...
Running test_device_not_found...
